In [1]:
import uchronia.uchronia_data_set as uds

In [2]:
import os

## -----------------------------------------------------------------------------
sample_data_dir = "/home/per202/data/documentation"
data_library_file = os.path.join(sample_data_dir, 'time_series_library.yaml')

In [3]:
## -----------------------------------------------------------------------------
data_library = uds.get_ensemble_dataset(data_library_file, '')
data_ids = uds.get_dataset_ids(data_library)
data_ids

['var1_obs',
 'var1_obs_collection',
 'var1_ens',
 'pet_fcast_ens',
 'rain_fcast_ens',
 'var1_fcast_ens',
 'var2_fcast_ens']

In [4]:
single_obs = uds.get_dataset(data_library, "var1_obs")

In [5]:
single_obs

CFFI pointer handle to a native pointer of type id "TIME_SERIES_PTR"

In [10]:
uds.as_xarray(single_obs)

<xarray.DataArray (ensemble: 1, time: 48)>
array([[ 1.1,  2.1,  3.1,  4.1,  5.1,  6.1,  7.1,  8.1,  9.1, 10.1, 11.1,
        12.1, 13.1, 14.1, 15.1, 16.1, 17.1, 18.1, 19.1, 20.1, 21.1, 22.1,
        23.1, 24.1, 25.1, 26.1, 27.1, 28.1, 29.1, 30.1, 31.1, 32.1, 33.1,
        34.1, 35.1, 36.1, 37.1, 38.1, 39.1, 40.1, 41.1, 42.1, 43.1, 44.1,
        45.1, 46.1, 47.1, 48.1]])
Coordinates:
  * ensemble  (ensemble) int64 0
  * time      (time) datetime64[ns] 2010-08-01T14:00:00 ... 2010-08-03T13:00:00

In [11]:
single_obs

CFFI pointer handle to a native pointer of type id "TIME_SERIES_PTR"

In [12]:
uds.as_xarray(single_obs)

<xarray.DataArray (ensemble: 1, time: 48)>
array([[ 1.1,  2.1,  3.1,  4.1,  5.1,  6.1,  7.1,  8.1,  9.1, 10.1, 11.1,
        12.1, 13.1, 14.1, 15.1, 16.1, 17.1, 18.1, 19.1, 20.1, 21.1, 22.1,
        23.1, 24.1, 25.1, 26.1, 27.1, 28.1, 29.1, 30.1, 31.1, 32.1, 33.1,
        34.1, 35.1, 36.1, 37.1, 38.1, 39.1, 40.1, 41.1, 42.1, 43.1, 44.1,
        45.1, 46.1, 47.1, 48.1]])
Coordinates:
  * ensemble  (ensemble) int64 0
  * time      (time) datetime64[ns] 2010-08-01T14:00:00 ... 2010-08-03T13:00:00

In [13]:
multi_obs = uds.get_dataset(data_library, "var1_obs_collection")
multi_obs

CFFI pointer handle to a native pointer of type id "ENSEMBLE_PTR_TIME_SERIES_PTR"

In [9]:
import uchronia.uchronia_time_series as uts
station_ids = uts.sub_identifiers(data_library, "var1_obs_collection")
print(station_ids)

['123', '456']


In [14]:
uds.as_xarray(multi_obs)

<xarray.DataArray (ensemble: 2, time: 48)>
array([[ 1.1,  2.1,  3.1,  4.1,  5.1,  6.1,  7.1,  8.1,  9.1, 10.1, 11.1,
        12.1, 13.1, 14.1, 15.1, 16.1, 17.1, 18.1, 19.1, 20.1, 21.1, 22.1,
        23.1, 24.1, 25.1, 26.1, 27.1, 28.1, 29.1, 30.1, 31.1, 32.1, 33.1,
        34.1, 35.1, 36.1, 37.1, 38.1, 39.1, 40.1, 41.1, 42.1, 43.1, 44.1,
        45.1, 46.1, 47.1, 48.1],
       [ 1.2,  2.2,  3.2,  4.2,  5.2,  6.2,  7.2,  8.2,  9.2, 10.2, 11.2,
        12.2, 13.2, 14.2, 15.2, 16.2, 17.2, 18.2, 19.2, 20.2, 21.2, 22.2,
        23.2, 24.2, 25.2, 26.2, 27.2, 28.2, 29.2, 30.2, 31.2, 32.2, 33.2,
        34.2, 35.2, 36.2, 37.2, 38.2, 39.2, 40.2, 41.2, 42.2, 43.2, 44.2,
        45.2, 46.2, 47.2, 48.2]])
Coordinates:
  * ensemble  (ensemble) int64 0 1
  * time      (time) datetime64[ns] 2010-08-01T14:00:00 ... 2010-08-03T13:00:00